In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

In [8]:
# Load dataset
df = pd.read_csv(r'C:\Users\Muneeeb\Desktop\project\PROJECTS\final_merged_dataset_updated.csv')

df = df.dropna(subset=['Modal Price (Rs./Quintal)'])  # Drop rows with missing target

df['Price Date'] = pd.to_datetime(df['Price Date'], errors='coerce')  # Convert date column if the value is other than numeric value
# it coverts into 
# # NAN
df = df.dropna(subset=['Price Date'])  # Remove rows with invalid dates

df['year'] = df['Price Date'].dt.year  # Extract year
df['month'] = df['Price Date'].dt.month  # Extract month


C:\Users\Muneeeb\AppData\Local\Temp\ipykernel_7148\3195079791.py:6: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Price Date'] = pd.to_datetime(df['Price Date'], errors='coerce')  # Convert date column if the value is other than numeric value


In [103]:
df.head()

,District Name,Market Name,Commodity,Variety,Grade,Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Price Date,Temperature,year,month
0,Jammu,Akhnoor,Spinach,Other,FAQ,600.0,1000.0,800.0,2020-12-19,-3.68,2020,12
1,Jammu,Akhnoor,Spinach,Other,FAQ,700.0,900.0,800.0,2021-02-10,2.87,2021,2
2,Jammu,Akhnoor,Spinach,Other,FAQ,800.0,1000.0,900.0,2021-03-01,0.65,2021,3
3,Jammu,Akhnoor,Spinach,Other,FAQ,800.0,1000.0,900.0,2021-01-02,-2.69,2021,1
4,Jammu,Akhnoor,Spinach,Other,FAQ,800.0,1000.0,900.0,2021-03-02,2.75,2021,3


Reasons for Using Label Encoding in Your Case:
✅ 1. Random Forest Requires Numeric Input

Algorithms like Random Forest can’t work with text data.

Label Encoding assigns each category a unique number so the model can use them.

✅ 2. Categorical Features Have No Ordinal Relationship

Since features like Market Name and Commodity are just categories, one-hot encoding (OHE) could be an alternative.

But OHE creates too many columns (memory-heavy), so Label Encoding is better when you have many unique categories.
srinagar budgam ganderbal
1           0    0
0           1    0
0           0    1

le.fit(df[col])  # Step 1: Learn unique categories
df[col] = le.transform(df[col])  # Step 2: Replace text with numbers

Step 1: Learning Unique Categories (.fit())
When we run:
le = LabelEncoder()  
le.fit(df['Commodity'])
👉 The Label Encoder scans the column and finds the unique categories:

Apple → 0

Pea → 1

Walnut → 2

✅ At this point, the encoder remembers these mappings.
Step 2: Replacing Text with Numbers (.transform())
Now, when we run:
df['Commodity'] = le.transform(df['Commodity'])
👉 It replaces each value in the column with its assigned number:

Commodity (Original)	Commodity (Encoded)
Apple	0
Walnut	2
Pea	1
Walnut	2
✅ The column is now numeric, ready for use in a machine learning model!

In [9]:
# Encode categorical variables
categorical_cols = ['District Name', 'Market Name', 'Commodity', 'Variety', 'Grade']
le_dict = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    le_dict[col] = le



In [96]:
le_dict

{'District Name': LabelEncoder(),
 'Market Name': LabelEncoder(),
 'Commodity': LabelEncoder(),
 'Variety': LabelEncoder(),
 'Grade': LabelEncoder()}

In [10]:
df['District Name']


0         3
1         3
2         3
3         3
4         3
         ..
102201    9
102202    9
102203    9
102204    9
102205    9
Name: District Name, Length: 102206, dtype: int32

In [11]:
for col, le in le_dict.items():
    print(f"Column: {col}")
    print(dict(enumerate(le.classes_)))  # Encoding mapping dikhega
    print()

Column: District Name
{0: 'Anantnag', 1: 'Badgam', 2: 'Baramulla', 3: 'Jammu', 4: 'Kathua', 5: 'Kupwara', 6: 'Pulwama', 7: 'Rajouri', 8: 'Srinagar', 9: 'Udhampur'}

Column: Market Name
{0: 'Akhnoor', 1: 'Ashahipora (Anantnagh)', 2: 'Batote', 3: 'Bumhama-Kupwara (F&V)', 4: 'Ganderbal', 5: 'Kanispora Baramulla (F&V)', 6: 'Kathua', 7: 'Kulgam', 8: 'Narwal Jammu (F&V)', 9: 'Nowpora', 10: 'Parimpore', 11: 'Pulwama (F&V)', 12: 'Rajouri (F&V)', 13: 'Reasi', 14: 'Samba', 15: 'Shopian', 16: 'Udhampur', 17: 'Zaloosa-Chararishrief (F&V)'}

Column: Commodity
{0: 'Apple', 1: 'Apricot(Jardalu/Khumani)', 2: 'Banana', 3: 'Beetroot', 4: 'Bhindi(Ladies Finger)', 5: 'Cabbage', 6: 'Carrot', 7: 'Cauliflower', 8: 'Cherry', 9: 'Cucumbar(Kheera)', 10: 'Garlic', 11: 'Ginger(Dry)', 12: 'Ginger(Green)', 13: 'Grapes', 14: 'Little gourd (Kundru)', 15: 'Mashrooms', 16: 'Onion', 17: 'Peach', 18: 'Pear(Marasebu)', 19: 'Plum', 20: 'Pomegranate', 21: 'Potato', 22: 'Spinach', 23: 'Turnip', 24: 'Walnut'}

Column: Variety

In [12]:
df.head()

,District Name,Market Name,Commodity,Variety,Grade,Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Price Date,Temperature,year,month
0,3,0,22,38,2,600.0,1000.0,800.0,2020-12-19,-3.68,2020,12
1,3,0,22,38,2,700.0,900.0,800.0,2021-02-10,2.87,2021,2
2,3,0,22,38,2,800.0,1000.0,900.0,2021-03-01,0.65,2021,3
3,3,0,22,38,2,800.0,1000.0,900.0,2021-01-02,-2.69,2021,1
4,3,0,22,38,2,800.0,1000.0,900.0,2021-03-02,2.75,2021,3


In [13]:
df.head()

,District Name,Market Name,Commodity,Variety,Grade,Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Price Date,Temperature,year,month
0,3,0,22,38,2,600.0,1000.0,800.0,2020-12-19,-3.68,2020,12
1,3,0,22,38,2,700.0,900.0,800.0,2021-02-10,2.87,2021,2
2,3,0,22,38,2,800.0,1000.0,900.0,2021-03-01,0.65,2021,3
3,3,0,22,38,2,800.0,1000.0,900.0,2021-01-02,-2.69,2021,1
4,3,0,22,38,2,800.0,1000.0,900.0,2021-03-02,2.75,2021,3


In [14]:
# Prepare data
X = df[['District Name', 'Market Name', 'Commodity', 'Variety', 'Grade', 'year', 'month', 'Temperature']]
y = df['Modal Price (Rs./Quintal)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Train-test split

tuning is not making that much impact in model

# Hyperparameter tuning using RandomizedSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],  
    'max_depth': [10, 20, 30, None],  
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [1, 2, 4]
}

rf_random = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_distributions=param_grid,
    n_iter=10,
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)
rf_random.fit(X_train, y_train)

best_params = rf_random.best_params_
print("Best Parameters:", best_params)

In [15]:
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

RAE = √(1/n * Σ |yᵢ - ŷᵢ|)

In [16]:
# Evaluate model
y_pred = model.predict(X_test)
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R2 Score:", r2_score(y_test, y_pred))


MAE: 464.87333162583104
RMSE: 981.3246720538903
R2 Score: 0.9563877581616788


In [17]:

import numpy as np

mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
accuracy = 100 - mape  # Accuracy in %
print(f"Model Accuracy (based on MAPE): {accuracy:.2f}%")
print(mape)

Model Accuracy (based on MAPE): 87.52%
12.48155496890098


In [14]:
# Feature Importance
importances = model.feature_importances_
feature_names = X.columns
sorted_features = sorted(zip(importances, feature_names), reverse=True)
print("Feature Importances:", sorted_features)

Feature Importances: [(0.47484079526634937, 'Commodity'), (0.14980954115203382, 'Variety'), (0.12685677654658983, 'year'), (0.09915267584675386, 'Grade'), (0.0654807102375801, 'month'), (0.040938355070916616, 'Temperature'), (0.02257937793651607, 'District Name'), (0.020341767943260255, 'Market Name')]


In [6]:
# Dictionary of valid values
valid_values = {}
for col in categorical_cols:
    valid_values[col] = le_dict[col].classes_

def validate_sample_strict(sample):
    if sample['Market Name'] not in valid_values['Market Name']:
        print(f"❌ Market '{sample['Market Name']}' not found.")
        print(f"✅ Valid Markets: {valid_values['Market Name']}")
        return False
    
    combo_exists = df[
        (df['Commodity'] == le_dict['Commodity'].transform([sample['Commodity']])[0]) &
        (df['Variety'] == le_dict['Variety'].transform([sample['Variety']])[0]) &
        (df['Grade'] == le_dict['Grade'].transform([sample['Grade']])[0])
    ]
    if combo_exists.empty:
        print(f"❌ Combination of Commodity '{sample['Commodity']}', Variety '{sample['Variety']}', and Grade '{sample['Grade']}' does not exist.")
        return False
    
    print("✅ Sample is valid for prediction!")
    return True

def predict_price(sample):
    sample_encoded = sample.copy()
    for col in categorical_cols:
        sample_encoded[col] = le_dict[col].transform([sample[col]])[0]

    sample_encoded['year'] = sample['Price Date'].year
    sample_encoded['month'] = sample['Price Date'].month

    model_input = pd.DataFrame([{
        'District Name': sample_encoded['District Name'],
        'Market Name': sample_encoded['Market Name'],
        'Commodity': sample_encoded['Commodity'],
        'Variety': sample_encoded['Variety'],
        'Grade': sample_encoded['Grade'],
        'year': sample_encoded['year'],
        'month': sample_encoded['month'],
        'Temperature': sample['Temperature']
    }])

    prediction = model.predict(model_input)[0]
    return prediction

In [19]:
# Example Sample
sample = {
    'Commodity': 'Apple',
    'Variety': 'American',
    'Grade': 'Medium',
    'District Name': 'Srinagar',
    'Market Name': 'Ganderbal',
    'Price Date': pd.to_datetime('2024-03-18'),
    'Temperature': 5.6
}

if validate_sample_strict(sample):
    predicted_price = predict_price(sample)
    print("Predicted Modal Price:", predicted_price)
else:
    print("⚠️ Invalid input, cannot predict.")

✅ Sample is valid for prediction!
Predicted Modal Price: 7148.01


In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# import numpy as np

# # Load dataset
# df = pd.read_csv(r'C:\Users\91979\Desktop\agriAPI\final_merged_dataset (1).csv')


# # Drop rows with missing target
# df = df.dropna(subset=['Modal Price (Rs./Quintal)'])

# # Extract date features
# df['Price Date'] = pd.to_datetime(df['Price Date'], errors='coerce')
# df = df.dropna(subset=['Price Date'])
# df['year'] = df['Price Date'].dt.year
# df['month'] = df['Price Date'].dt.month

# # Encode categorical variables
# categorical_cols = ['District Name', 'Market Name', 'Commodity', 'Variety', 'Grade']
# le_dict = {}
# for col in categorical_cols:
#     le = LabelEncoder()
#     df[col] = le.fit_transform(df[col])
#     le_dict[col] = le

# # Prepare data
# X = df[['District Name', 'Market Name', 'Commodity', 'Variety', 'Grade', 'year', 'month', 'Temperature']]
# y = df['Modal Price (Rs./Quintal)']

# # Train-test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Hyperparameter tuning using RandomizedSearchCV
# param_grid = {
#     'n_estimators': [50, 100, 200],  
#     'max_depth': [10, 20, 30, None],  
#     'min_samples_split': [2, 5, 10],  
#     'min_samples_leaf': [1, 2, 4]
# }

# rf_random = RandomizedSearchCV(
#     estimator=RandomForestRegressor(random_state=42),
#     param_distributions=param_grid,
#     n_iter=10,                                      # Faster tuning
#     cv=3,
#     verbose=2,
#     random_state=42,
#     n_jobs=-1  # Parallel processing
# )

# rf_random.fit(X_train, y_train)
# best_params = rf_random.best_params_
# print("Best Parameters:", best_params)

# # Train model with best parameters
# model = RandomForestRegressor(**best_params, random_state=42)
# model.fit(X_train, y_train)

# # Evaluation
# y_pred = model.predict(X_test)
# print("MAE:", mean_absolute_error(y_test, y_pred))
# print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
# print("R2 Score:", r2_score(y_test, y_pred))

# # Feature Importance
# importances = model.feature_importances_
# feature_names = X.columns
# sorted_features = sorted(zip(importances, feature_names), reverse=True)
# print("Feature Importances:", sorted_features)

# # Create valid values dictionary
# valid_values = {}
# for col in categorical_cols:
#     valid_values[col] = le_dict[col].classes_

# def validate_sample_strict(sample):
#     if sample['Market Name'] not in valid_values['Market Name']:
#         print(f"❌ Market '{sample['Market Name']}' not found.")
#         print(f"✅ Valid Markets: {valid_values['Market Name']}")
#         return False

#     combo_exists = df[
#         (df['Commodity'] == le_dict['Commodity'].transform([sample['Commodity']])[0]) &
#         (df['Variety'] == le_dict['Variety'].transform([sample['Variety']])[0]) &
#         (df['Grade'] == le_dict['Grade'].transform([sample['Grade']])[0])
#     ]
#     if combo_exists.empty:
#         print(f"❌ Combination of Commodity '{sample['Commodity']}', Variety '{sample['Variety']}', and Grade '{sample['Grade']}' does not exist.")
#         return False

#     print("✅ Sample is valid for prediction!")
#     return True

# def predict_price(sample):
#     sample_encoded = sample.copy()
#     for col in categorical_cols:
#         sample_encoded[col] = le_dict[col].transform([sample[col]])[0]

#     # Extract year and month separately
#     sample_encoded['year'] = sample['Price Date'].year
#     sample_encoded['month'] = sample['Price Date'].month

#     # Only keep the columns used in training
#     model_input = pd.DataFrame([{
#         'District Name': sample_encoded['District Name'],
#         'Market Name': sample_encoded['Market Name'],
#         'Commodity': sample_encoded['Commodity'],
#         'Variety': sample_encoded['Variety'],
#         'Grade': sample_encoded['Grade'],
#         'year': sample_encoded['year'],
#         'month': sample_encoded['month'],
#         'Temperature': sample['Temperature']
#     }])

#     prediction = model.predict(model_input)[0]
#     return prediction

# # Example
# sample = {
#     'Commodity': 'Apple',
#     'Variety': 'American',
#     'Grade': 'Medium',
#     'District Name': 'Srinagar',
#     'Market Name': 'Ganderbal',
#     'Price Date': pd.to_datetime('2024-03-18'),
#     'Temperature': 5.6
# }

# if validate_sample_strict(sample):
#     predicted_price = predict_price(sample)
#     print("Predicted Modal Price:", predicted_price)
# else:
#     print("⚠️ Invalid input, cannot predict.")


In [7]:
def validate_sample_strict(sample):
    # Market check
    if sample['Market Name'] not in valid_values['Market Name']:
        print(f"❌ Market '{sample['Market Name']}' not found in dataset.")
        print(f"✅ Valid Markets: {valid_values['Market Name']}")
        return False
    
    # Check if District and Market Name combination exists
    district_encoded = le_dict['District Name'].transform([sample['District Name']])[0]
    market_encoded = le_dict['Market Name'].transform([sample['Market Name']])[0]
    
    combo_exists = df[
        (df['District Name'] == district_encoded) & 
        (df['Market Name'] == market_encoded)
    ]

    if combo_exists.empty:
        print(f"❌ Market '{sample['Market Name']}' does not exist in District '{sample['District Name']}'.")
        return False

    print("✅ Sample is valid for prediction!")
    return True


In [5]:
import matplotlib.pyplot as plt
import numpy as np

feature_importance = model.feature_importances_  # Model ka attribute
feature_names = X_train.columns  # Features ke naam

# Feature importance plot
sorted_idx = np.argsort(feature_importance)
plt.figure(figsize=(10, 5))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align="center")
plt.yticks(range(len(sorted_idx)), np.array(feature_names)[sorted_idx])
plt.xlabel("Feature Importance")
plt.title("Feature Importance in Price Prediction")
plt.show()


NameError: name 'model' is not defined

In [22]:
pip install matplotlib


Note: you may need to restart the kernel to use updated packages.


In [4]:
import matplotlib.pyplot as plt
import numpy as np

feature_importance = model.feature_importances_  # Model ka attribute
feature_names = X_train.columns  # Features ke naam

# Feature importance plot
sorted_idx = np.argsort(feature_importance)
plt.figure(figsize=(10, 5))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align="center")
plt.yticks(range(len(sorted_idx)), np.array(feature_names)[sorted_idx])
plt.xlabel("Feature Importance")
plt.title("Feature Importance in Price Prediction")
plt.show()


NameError: name 'model' is not defined

In [24]:
!pip install seaborn --quiet


In [3]:
sample = {
    'Commodity': 'Apple',
    'Variety': 'American',
    'Grade': 'Small',
    'District Name': 'Srinagar',
    'Market Name': 'Ganderbal',
    'Price Date': pd.to_datetime('2027-03-18'),
    'Temperature': 1
}

if validate_sample_strict(sample):
    predicted_price = predict_price(sample)
    print("Predicted Modal Price:", predicted_price)
else:
    print("⚠️ Invalid input, cannot predict.")

NameError: name 'pd' is not defined

In [19]:
import pickle

# Save trained model
with open("model.pkl", "wb") as model_file:
    pickle.dump(model, model_file)

print("✅ Model saved successfully as model.pkl")


✅ Model saved successfully as model.pkl


In [30]:
import pickle

# Save label encoders dictionary
with open("label_encoders.pkl", "wb") as le_file:
    pickle.dump(le_dict, le_file)

print("✅ Label encoders saved successfully as label_encoders.pkl")


✅ Label encoders saved successfully as label_encoders.pkl
